In [129]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import xgboost as xgb
import pickle

In [130]:

df = pd.read_csv('fusionV3.csv',
                   sep=';',
                   engine='python',
                   quoting=3,
                   on_bad_lines='skip',
                   encoding='utf-8')

df_clean = df.copy()

In [131]:
def clean_value(value):
    # Si la valeur est NaN ou une chaîne indiquant une valeur manquante
    if pd.isna(value) or value in ['Non disponible', '?', '? $', '- $', '-']:
        return np.nan
    
    # Si c'est une chaîne, on la nettoie et la convertit
    if isinstance(value, str):
        # Supprimer les symboles $, espaces, et remplacer les virgules par des points
        cleaned_value = value.replace('$', '').replace(' ', '').replace(',', '.')
        
        # Si après nettoyage, la chaîne est vide ou juste un tiret
        if cleaned_value == '' or cleaned_value == '-':
            return np.nan
        
        try:
            return float(cleaned_value)
        except ValueError:
            # Si la conversion échoue, on retourne NaN
            return np.nan
    
    # Si ce n'est pas une chaîne, on retourne la valeur telle quelle
    return value

def extract_minutes(duree):
    # Si la valeur est NaN ou non-string, retourner NaN
    if pd.isna(duree):
        return np.nan
    
    # Convertir en string si c'est un nombre
    if not isinstance(duree, str):
        try:
            return float(duree)  # Si c'est déjà un nombre, le retourner tel quel
        except (ValueError, TypeError):
            return np.nan
    
    # Chercher le format "1h 32min"
    match = re.search(r'(\d+)h\s*(\d+)min', duree)
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        return hours * 60 + minutes
    
    # Chercher seulement les minutes
    match = re.search(r'(\d+)\s*min', duree)
    if match:
        return int(match.group(1))
    
    # Chercher seulement les heures
    match = re.search(r'(\d+)h', duree)
    if match:
        return int(match.group(1)) * 60
    
    # Si c'est juste un nombre
    match = re.search(r'(\d+)', duree)
    if match:
        return int(match.group(1))
    
    return np.nan

def clean_note_moyenne(value):
    if pd.isna(value) or value in ['Non disponible', 'note_moyenne']:
        return np.nan
    
    # Si c'est une chaîne, remplacer les virgules par des points
    if isinstance(value, str):
        value = value.replace(',', '.')
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return np.nan
    

def clean_monetary_value(value):
    """Nettoie et convertit les valeurs monétaires avec gestion améliorée des erreurs"""
    # Dictionnaire des taux de conversion approximatifs
    currency_rates = {
        '€': 1.1,  # Euro vers Dollar
        '£': 1.25, # Livre vers Dollar
        '¥': 0.0068, # Yen vers Dollar
        'CA$': 0.73, # Dollar canadien vers Dollar américain
        'A$': 0.65  # Dollar australien vers Dollar américain
    }
    
    if pd.isna(value) or value in ['Non disponible', '?', '? $', '- $', '-', '', 'nan', '? €']:
        return np.nan
        
    if isinstance(value, str):
        # Vérifier si la chaîne est un titre de film ou un texte non pertinent
        # Si la longueur est supérieure à 20 caractères et ne contient pas de chiffres, c'est probablement un titre
        if len(value) > 20 and not any(c.isdigit() for c in value):
            return np.nan
            
        # Détecter la devise et convertir en USD
        for symbol, rate in currency_rates.items():
            if symbol in value:
                cleaned_value = value.replace(symbol, '').replace(' ', '').strip()
                try:
                    return float(cleaned_value.replace(',', '.')) * rate
                except ValueError:
                    continue
                    
        # Traitement standard pour les montants en dollars
        cleaned_value = value.replace('$', '').replace(' ', '').replace(',', '.').strip()
        
        # Si après nettoyage, la chaîne est vide ou juste un tiret
        if cleaned_value == '' or cleaned_value == '-':
            return np.nan
            
        try:
            return float(cleaned_value)
        except ValueError:
            # Si ce n'est pas un nombre, c'est probablement un texte non pertinent
            return np.nan
            
    return value

    


In [132]:
# Nettoyage des colonnes monétaires
numeric_columns = ['budget', 'box_office_demarrage', 'year', 'trailer_views',]

# monetary_columns = ['budget', 'box_office_demarrage', 'box_office_france', 
#                         'recette_usa', 'recette_monde']

for col in numeric_columns:
     if col in df_clean.columns:
         df_clean[col] = df_clean[col].apply(clean_monetary_value)

In [133]:
#  Nettoyage de la durée
if 'duration' in df_clean.columns:
    df_clean['duree'] = df_clean['duration'].apply(extract_minutes)

In [134]:
# 6. Nettoyage de la note moyenne
if 'note_moyenne' in df_clean.columns:
    df_clean['note_moyenne'] = df_clean['note_moyenne'].apply(clean_note_moyenne)
elif 'viewer_rating' in df_clean.columns:
    df_clean['note_moyenne'] = df_clean['viewer_rating'].apply(clean_note_moyenne)
else:
    print("Aucune colonne de note moyenne trouvée")

In [135]:

        
def extract_month(date_str):
    """Extrait le mois à partir d'une date au format DD/MM/YYYY"""
    if pd.isna(date_str):
        return np.nan
        
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', str(date_str))
    if match:
        return int(match.group(2))
        
    return np.nan

def extract_year(date_str):
    """Extrait l'année à partir d'une date au format DD/MM/YYYY"""
    if pd.isna(date_str):
        return np.nan
        
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})', str(date_str))
    if match:
        return int(match.group(3))
    
    # Format YYYY uniquement
    match = re.search(r'^(\d{4})$', str(date_str).strip())
    if match:
        return int(match.group(1))
        
    return np.nan

def determine_season(month):
    """Détermine la saison en fonction du mois"""
    if pd.isna(month):
        return np.nan
    month = int(month)
    if month in [12, 1, 2]:
        return 1  # Hiver
    elif month in [3, 4, 5]:
        return 2  # Printemps
    elif month in [6, 7, 8]:
        return 3  # Été
    else:
        return 4  # Automne

def is_holiday_season(month, day=15):
    """Détermine si c'est une période de vacances scolaires"""
    if pd.isna(month):
        return np.nan
    
    month = int(month)
    
    # Vacances d'été (juillet-août)
    if month in [7, 8]:
        return 1
    # Vacances de Noël (décembre)
    elif month == 12:
        return 1
    # Vacances d'hiver (février)
    elif month == 2:
        return 1
    # Vacances de printemps (avril)
    elif month == 4:
        return 1
    # Vacances de la Toussaint (octobre)
    elif month == 10:
        return 1
    else:
        return 0
    

In [136]:
# Extraire les infos temporelles
df_clean['annee_sortie'] = df_clean['date_sortie_france'].apply(extract_year)
df_clean['mois_sortie'] = df_clean['date_sortie_france'].apply(extract_month)
df_clean['saison_sortie'] = df_clean['mois_sortie'].apply(determine_season)
df_clean['vacances_scolaires'] = df_clean['mois_sortie'].apply(is_holiday_season)
        
# Créer des indicateurs pour les périodes clés de sortie
df_clean['sortie_ete'] = (df_clean['mois_sortie'].isin([6, 7, 8])).astype(int)
df_clean['sortie_fetes'] = (df_clean['mois_sortie'].isin([11, 12])).astype(int)
        

df_clean['post_covid'] = (df_clean['annee_sortie'] >= 2020).astype(int)
        

In [137]:
def categorize_budget(budget):
        if pd.isna(budget):
            return np.nan
        elif budget < 10000000:  # Moins de 10 millions
            return 1  # Petit budget
        elif budget < 50000000:  # Entre 10 et 50 millions
            return 2  # Budget moyen
        elif budget < 100000000:  # Entre 50 et 100 millions
            return 3  # Gros budget
        else:  # 100 millions et plus
            return 4  # Blockbuster

# Transformation logarithmique du budget
df_clean['log_budget'] = np.log1p(df_clean['budget'])

#Catégorisation du budget    
df_clean['categorie_budget'] = df_clean['budget'].apply(categorize_budget)


In [138]:
if 'top_stars' in df_clean.columns:
        df_clean['star_count'] = df_clean['top_stars'].apply(
            lambda x: len(str(x).split(',')) if not pd.isna(x) else 0
        )
        
        # Caractéristique pour les films avec des stars importantes
        famous_actors = ['Brad Pitt', 'Leonardo DiCaprio', 'Tom Cruise', 'Dwayne Johnson', 
                         'Jennifer Lawrence', 'Scarlett Johansson', 'Robert Downey Jr',
                         'Gérard Depardieu', 'Jean Dujardin', 'Omar Sy', 'Marion Cotillard']
        
        df_clean['has_famous_actor'] = df_clean['top_stars'].apply(
            lambda x: 1 if isinstance(x, str) and any(actor.lower() in str(x).lower() for actor in famous_actors) else 0
        )

In [139]:
categorical_columns = []
if 'genre_principale' in df_clean.columns:
    categorical_columns.append('genre_principale')
if 'saison_sortie' in df_clean.columns:
    categorical_columns.append('saison_sortie')
if 'categorie_budget' in df_clean.columns:
    categorical_columns.append('categorie_budget')
if 'mois_sortie' in df_clean.columns:
    categorical_columns.append('mois_sortie')
if 'vacances_scolaires' in df_clean.columns:
    categorical_columns.append('vacances_scolaires')
if 'sortie_ete' in df_clean.columns:
    categorical_columns.append('sortie_ete')
if 'sortie_fetes' in df_clean.columns:
    categorical_columns.append('sortie_fetes')
if 'post_covid' in df_clean.columns:
    categorical_columns.append('post_covid')



# One-hot encoding
if categorical_columns:
    df_encoded = pd.get_dummies(df_clean, columns=categorical_columns, drop_first=False)
    
    genre_columns = [col for col in df_encoded.columns if col.startswith('genre_principale_')]
    saison_columns = [col for col in df_encoded.columns if col.startswith('saison_sortie_')]
    budget_cat_columns = [col for col in df_encoded.columns if col.startswith('categorie_budget_')]
    
    print(f"Colonnes de genre créées: {genre_columns}")
    print(f"Colonnes de saison créées: {saison_columns}")
    print(f"Colonnes de catégorie budget créées: {budget_cat_columns}")
else:
    print("Aucune colonne catégorielle trouvée pour l'encodage")
    df_encoded = df_clean.copy()
    genre_columns = []
    saison_columns = []
    budget_cat_columns = []

Colonnes de genre créées: ['genre_principale_ Antonio, le si mielleux objet de l’affection naissante de Margo, et Eduardo Perez, le père d’Antonio, propriétaire du restaurant Salsa & Salsa et l’homme qui se cache peut-être derrière le masque d’El Macho, le plus impitoyable et, comme son nom l’indique, méchant macho que la terre ait jamais porté."', 'genre_principale_ Dev’Reaux, le liftier novice et Odessa, la femme de ménage belliqueuse, jouissent néanmoins d’un atout majeur : ils connaissent le bâtiment de fond en comble. Sans jamais s’en être rendu compte, ils repèrent les lieux du crime depuis des années."', 'genre_principale_ Roscoe Means, expert en explosifs ', "genre_principale_ enfin, Jeanne, prostituée, capable d'assassiner de sang froid.", 'genre_principale_- Artus | François Berléand', 'genre_principale_- Queen Latifah | Luc Besson', 'genre_principale_Adam Driver | Camille Cottin | Salma Hayek | Jeremy Irons | Jared Leto | Al Pacino | Ridley Scott', 'genre_principale_Adam San

In [140]:
print("Toutes les colonnes disponibles:")
for i, column in enumerate(df_encoded.columns):
    print(f"{i+1}. {column}")

# Afficher le nombre total de colonnes
print(f"\nNombre total de colonnes: {len(df_encoded.columns)}")

Toutes les colonnes disponibles:
1. film_id
2. titre_jpbox
3. genres
4. date_sortie_france
5. date_sortie_usa
6. duree_minutes
7. synopsis_x
8. realisateur
9. acteurs
10. pays_origine
11. budget
12. box_office_demarrage
13. box_office_france
14. recette_usa
15. recette_monde
16. image_url
17. note_moyenne
18. titre_clean
19. titre_allocine
20. film_url
21. film_image_url
22. release_date
23. duration
24. age_classification
25. producers
26. director
27. top_stars
28. press_rating
29. viewer_rating
30. languages
31. distributor
32. year_of_production
33. film_nationality
34. filming_secrets
35. fr_entry_week
36. us_entry_week
37. fr_entries
38. us_entries
39. awards
40. associated_genres
41. press_critics_count
42. viewer_critics_count
43. broadcast_category
44. trailer_views
45. synopsis_y
46. duree
47. annee_sortie
48. log_budget
49. star_count
50. has_famous_actor
51. genre_principale_ Antonio, le si mielleux objet de l’affection naissante de Margo, et Eduardo Perez, le père d’Antoni

In [141]:
# Sélection des features les plus pertinentes pour le modèle XGBoost

# Regroupement thématique des features
features_selection = {
    # 1. Features de base (durée et info film)
    'base': [col for col in df_encoded.columns if col in ['duree_minutes', 'duree', 'duree_film']],
    
    # 2. Features budgétaires (très importantes)
    'budget': [col for col in df_encoded.columns 
              if 'budget' in col.lower() 
              and 'budget_x_' not in col],
    
    # 3. Marketing (feature la plus importante selon l'analyse)
    'marketing': [col for col in df_encoded.columns if 'marketing' in col.lower()],
    
    # 4. Features temporelles
    'temporel': [
        # Sortie de base
        col for col in df_encoded.columns 
        if any(term in col for term in [
            'mois_sortie', 'annee_sortie', 'saison_sortie', 
            'vacances_scolaires', 'sortie_ete', 'sortie_fetes', 'post_covid'
        ]) or col.startswith('saison_sortie_') or col.startswith('mois_')
    ],
    
    # 5. Features de jour de sortie
    'jour_sortie': [
        col for col in df_encoded.columns 
        if 'wednesday' in col.lower() or 'weekend' in col.lower() or 'mercredi' in col.lower()
    ],
    
    # 6. Genre principal (catégorie la plus importante)
    'genre_principal': [col for col in df_encoded.columns if col.startswith('genre_principale_')],
    
    # 7. Genres associés
    'genre_associe': [
        col for col in df_encoded.columns 
        if col.startswith('genre_') and not col.startswith('genre_principale_')
        and not 'budget_x_genre' in col
    ],
    
    # 8. Interaction budget-genre (corrigé)
    'interaction': [col for col in df_encoded.columns if 'budget_x_' in col],
    
    # 9. Distributeur
    'distributeur': [
        col for col in df_encoded.columns 
        if col.startswith('distributor_') or col == 'is_major_studio'
    ],
    
    # 10. Origine et langue
    'origine': [col for col in df_encoded.columns 
               if col in ['is_english', 'is_french', 'is_usa', 'is_europe', 'is_asia', 'is_coproduction']],
    
    # 11. Acteurs et franchise
    'acteurs': [col for col in df_encoded.columns 
               if col in ['star_count', 'has_famous_actor', 'is_franchise']],
}

# Aplatir la liste de features
selected_features = []
for category, features in features_selection.items():
    # Filtrer uniquement les colonnes qui existent réellement
    existing_features = [f for f in features if f in df_encoded.columns]
    selected_features.extend(existing_features)
    print(f"{category}: {len(existing_features)} features")

# Éliminer les doublons (au cas où)
selected_features = list(set(selected_features))

print(f"\nNombre total de features sélectionnées: {len(selected_features)}")


base: 2 features
budget: 6 features
marketing: 0 features
temporel: 25 features
jour_sortie: 0 features
genre_principal: 836 features
genre_associe: 0 features
interaction: 0 features
distributeur: 0 features
origine: 0 features
acteurs: 2 features

Nombre total de features sélectionnées: 871


In [142]:
df_encoded.dtypes

film_id               object
titre_jpbox           object
genres                object
date_sortie_france    object
date_sortie_usa       object
                       ...  
sortie_ete_1            bool
sortie_fetes_0          bool
sortie_fetes_1          bool
post_covid_0            bool
post_covid_1            bool
Length: 914, dtype: object

In [143]:
# from sklearn.datasets import fetch_california_housing
# housing = fetch_california_housing()
# data = pd.DataFrame(housing.data, columns=housing.feature_names)
# data['target'] = housing.target

In [146]:
# Copier les données et définir X, y
data = df_encoded.copy()
X = data[selected_features]
y = data['box_office_demarrage']

# Convertir les chaînes numériques avec espaces en float (ex: "334 885" → 334885.0)
for col in data.select_dtypes(include='object').columns:
    try:
        # Test de conversion sur un échantillon
        data[col] = data[col].str.replace(' ', '').astype(float)
        print(f"Colonne convertie : {col}")
    except Exception:
        pass  # On ignore les colonnes non convertibles

# Mettre à jour X et y après tentative de conversion
X = data[selected_features]
y = data['box_office_demarrage']

# Nettoyage de y : suppression des valeurs NaN, infs, négatives
mask_valid_y = y.notna() & np.isfinite(y) & (y >= 0)
X, y = X[mask_valid_y], y[mask_valid_y]

# Convertir les colonnes non numériques restantes de X
for col in X.select_dtypes(include='object').columns:
    X[col] = pd.to_numeric(X[col], errors='coerce')
    if X[col].isna().any():
        X[col] = X[col].fillna(X[col].median())

# Appliquer log1p sur y (évite log(0))
y_log = np.log1p(y.clip(lower=0))

# Imputation des NaN et standardisation
X = SimpleImputer(strategy='median').fit_transform(X)
X = StandardScaler().fit_transform(X)

# Séparation train / test
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)
_, _, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [147]:

# Entraînement du modèle XGBoost
xgb_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    learning_rate=0.05,
    max_depth=8,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42
)
xgb_model.fit(X_train, y_train_log)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=0.1, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=200,
             n_jobs=None, num_parallel_tree=None, ...)

In [148]:

# Fonction d'évaluation du modèle
def evaluate_model(model, X, y_log, y_original, model_name="Modèle"):
    y_pred_log = model.predict(X)
    y_pred = np.expm1(y_pred_log)

    print(f"\nÉvaluation du {model_name}:")
    print(f"R² = {r2_score(y_original, y_pred):.4f}")
    print(f"R² (log) = {r2_score(y_log, y_pred_log):.4f}")
    print(f"MSE = {mean_squared_error(y_original, y_pred):.2f}")
    print(f"RMSE = {np.sqrt(mean_squared_error(y_original, y_pred)):.2f}")
    print(f"MAE = {mean_absolute_error(y_original, y_pred):.2f}")

    return y_pred

# Évaluation
_ = evaluate_model(xgb_model, X_test, y_test_log, y_test, "XGBoost")


Évaluation du XGBoost:
R² = 0.4545
R² (log) = 0.3547
MSE = 76637725597.04
RMSE = 276835.20
MAE = 118396.56
